In [4]:
import uproot
import matplotlib.pyplot as plt
import seaborn
import numpy as np
import math
from scipy.fft import fft, fftfreq
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import numpy as np
import pandas as pd
from datetime import datetime, timedelta

import os
from plotly.subplots import make_subplots
from plotly import tools
import plotly.offline as pyo
import sys
from tqdm import tqdm

In [6]:
#Opens log files that contain interaction information
with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_nu.log') as file:
    lines_nu = [line_nu.rstrip() for line_nu in file]
#with open('/Users/danielcarber/Documents/ICARUS/NuMI_Nue/1e1p/output_data.log') as file:
#    lines_data = [line_data.rstrip() for line_data in file]
print(lines_nu[1])

SELECTED,1,3,0,-1,1.000000,15.000000,7.000000,6.000000,7.000000,479.359890,394.468079,105209.111512,-1.000000,1,1,2,1,290.782806,694.850471,391.902618,346.974579,86027.953125,346.974579,2.185951,2.068959,0.961500,1.082368,0.000691,nan,86027.955271,1536.905309,-nan,-nan,3.141593,3.141593,0.594719,0.000000,0,0,1,


In [7]:
#Dictionaries to be filled in interaction information
#Signal is true interactions matched with reconstructed intercations
df_signal_nu = {'Type': [], 'Run': [], 'Evt': [], 'Subrun':[],'Nu_id':[],'Image_id':[],'Interaction_id':[], 'Category':[],
     'Category_topology':[],'category_interaction_mode':[],'leading_electron_ke_true':[], 'leading_electron_ke_reco':[],
     'leading_proton_ke_true':[],'leading_proton_ke_reco':[],'leading_electron_pid_true':[], 'leading_electron_pid_reco':[],
     'leading_proton_pid_true':[],'leading_proton_pid_reco':[],'visible_energy_true':[],'visible_energy_reco':[],'leading_electron_pt_true':[],
     'leading_electron_pt_reco':[],'leading_proton_pt_true':[],'leading_proton_pt_reco':[],'electron_angle_polar_angle_true':[],
     'leading_electron_polar_angle_reco':[],'electron_azimuthal_angle_true':[],'electron_azimuthal_angle_reco':[],
     'opening_angle_true':[],'opening_angle_reco':[],'interaction_pt_true':[],'interaction_pt_reco':[],
     'phiT_true':[],'phiT_reco':[],'alphaT_true':[],'alphaT_reco':[],'electron_softmax':[],'proton_softmax':[],
     'all_1e1p_cut':[],'all_1eNp_cut':[],'all_1eX_cut':[],'End':[]}

#Selected is reco interactions matched with true interactions
df_selected_nu = df_signal_nu
list_df_MC = ['Type', 'Run', 'Evt', 'Subrun','Nu_id','Image_id','Interaction_id', 'Category',
     'Category_topology','category_interaction_mode','leading_electron_ke_true', 'leading_electron_ke_reco',
     'leading_proton_ke_true','leading_proton_ke_reco','leading_electron_pid_true', 'leading_electron_pid_reco',
     'leading_proton_pid_true','leading_proton_pid_reco','visible_energy_true','visible_energy_reco','leading_electron_pt_true',
     'leading_electron_pt_reco','leading_proton_pt_true','leading_proton_pt_reco','electron_angle_polar_angle_true',
     'leading_electron_polar_angle_reco','electron_azimuthal_angle_true','electron_azimuthal_angle_reco',
     'opening_angle_true','opening_angle_reco','interaction_pt_true','interaction_pt_reco',
     'phiT_true','phiT_reco','alphaT_true','alphaT_reco','electron_softmax','proton_softmax','all_1e1p_cut','all_1eNp_cut','all_1eX_cut',"End"]


#Fills the dictionaries with data            
for line_nu in lines_nu:
    line_nu = line_nu.split(",")
    if line_nu[0] == 'SIGNAL':
        for i in range(len(line_nu)):
            df_signal_nu[list_df_MC[i]].append(line_nu[i])
    elif line_nu[0] == 'SELECTED':
        for i in range(len(line_nu)):
            df_selected_nu[list_df_MC[i]].append(line_nu[i])

In [22]:
signal,sel_1e, sel_1e1pi1p, sel_nue_other, sel_NC, sel_cosmic,sel_numu = [],[],[],[],[],[],[]
total_events =0
for index, category in enumerate(df_selected_nu['Category_topology']):
    if df_selected_nu['all_1e1p_cut'][index] == '1' or df_selected_nu['all_1eNp_cut'][index] == '1':
        print(df_selected_nu['visible_energy_reco'][index])
        total_events +=1
        if (category[0] == '0' or category[0] == '2'):
            signal.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '1':
            sel_1e.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '3':
            sel_1e1pi1p.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '4':
            sel_nue_other.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '5':
            sel_NC.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '6':
            sel_cosmic.append(float(df_selected_nu['visible_energy_reco'][index]))
        elif category[0] == '7':
            sel_numu.append(float(df_selected_nu['visible_energy_reco'][index]))
        

711.629590
711.629590
1108.988253
1108.988253
1017.363093
1175.163108
428.537513
438.143979
1250.944569
1250.944569
286.675766
724.527936
835.750094
835.750094
280.960377
1230.275117
1230.275117
696.267024
2930.538565
2930.538565
935.974570
675.902274
675.902274
773.693440
773.693440
1109.709009
1109.709009
345.147191
345.147191
3189.723311
3189.723311
186.821555
293.623961
216.634169
217.179876
437.747214
437.747214
664.893325
789.049312
789.049312
595.254048
650.895633
650.895633
526.733119
526.733119
219.829936
664.164420
1740.860680
1474.028093
1474.028093
740.903958
860.368808
860.368808
455.351589
530.979154
2332.237052
2186.069819
2186.069819
767.324528
1713.494049
1713.494049
187.959442
1023.997817
1023.997817
629.860433
1170.225631
2543.326032
2543.326032
999.320766
1219.202420
553.665796
1582.036712
1582.036712
542.448384
542.448384
291.420966
291.420966
292.945777
517.115070
1388.885560
1388.885560
361.516349
361.516349
1651.991862
1651.991862
732.123093
428.951902
1903.6080

In [23]:
def hist_1d(metric, bounds,bins, xaxis_name, Plot_title, selection):
    fig = make_subplots(rows=1,cols=1,subplot_titles = (f"<span style='font-size: 42px;'>{Plot_title}",))
    
    
    print('Purity: ', len(signal)/total_events)
    print('Number of signal: ',len(signal),' Number of selected:', total_events)
    cosmic_size = len(sel_cosmic)
    numu_size = len(sel_numu)
    NC_size = len(sel_NC)
    signal_size = len(signal)
    nue_size = len(sel_nue_other)
    
    
    fig.add_trace(go.Histogram(x=sel_cosmic,marker_color = 'yellow',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                               name = f'Cosmics ({cosmic_size}, {100*cosmic_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_numu,marker_color = 'blue',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'\u03BD<sub>\u03BC</sub> ({numu_size}, {100*numu_size/total_events:.2f}%)'),row = 1, col =1)
    fig.add_trace(go.Histogram(x=sel_NC,marker_color = 'black',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'NC ({NC_size}, {100*NC_size/total_events:.2f}%)'),row = 1, col =1)
    
    if selection != 'Nue':
        #nue_size =  len(df[metric][mask_other_nue][reco_signal][mask])
        fig.add_trace(go.Histogram(x=sel_nue_other,marker_color = 'orange',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                                  name = f'Other \u03BD<sub>e</sub> ({nue_size}, {100*nue_size/total_events:.2f}%)'),row = 1, col =1)
        fig.add_trace(go.Histogram(x=signal,marker_color = 'darkgreen',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'{selection} ({signal_size}, {100*signal_size/total_events:.2f}%)'),row = 1, col =1)
    else:
        fig.add_trace(go.Histogram(x=signal,marker_color = 'darkgreen',xbins=dict(start = bins[0],end = bins[1],size=bins[2]),
                              name = f'\u03BD<sub>e</sub> ({signal_size}, {100*signal_size/total_events:.2f}%)'),row = 1, col =1)
    
    
    fig.update_xaxes(title_text = xaxis_name,row = 1, col = 1)
    fig.update_yaxes(title_text = "Entries",row = 1, col = 1,)
    fig.update_layout(font = dict(size=26),legend=dict(
        yanchor="top",
        y=0.99,
        xanchor="left",
        x=0.43,
        bgcolor="LightSteelBlue",
            bordercolor="Black",
    ))
    fig.update_annotations(font_size=46)
    fig.update_layout(barmode='stack')
    fig.update_layout(xaxis = dict(range = bounds))
    fig.update_layout(height = 900, width = 900,showlegend = True)
    fig.show()


In [24]:
#hist_1d('reco_nu_energy',[0,5000],[0,5000,250],'Reco Neutrino Energy [MeV]','Selected 1e1p Events','1e1p')
hist_1d('reco_nu_energy',[0,5000],[0,5000,250],'Reco Neutrino Energy [MeV]','Selected 1eNp Events','1eNp')
#hist_1d('reco_nu_energy',[0,5000],[0,5000,100],'Reco Neutrino Energy [MeV]','Selected CC \u03BD<sub>e</sub> Inclusive Events','Nue')

Purity:  0.4626334519572954
Number of signal:  130  Number of selected: 281
